In [8]:
import sys
sys.path.insert(1, '../..')
import os
import re
import pandas as pd
from collections import Counter
from medicine import config
from medicine.script.data_process import DataProcess

In [33]:
file_path = os.path.join(config.Path['data_dir'], '本草纲目(簡體)_Ver2.csv')
data = pd.read_csv(file_path, index_col=[0], engine='python', encoding='utf-8-sig')
data.columns

Index(['目錄', '篇名', '釋名', '內容', '部位', '部位內容', '主治', '氣味', '附方'], dtype='object')

In [37]:
compounds = data.loc[:,'附方']
compounds[11:16]

11    旧一，新一。辟禳时疫∶半天河水，饮之。（《医林集要》）身体白驳∶取树木 孔中水洗之，捣桂末唾...
12                                                  NaN
13    新三。目不得瞑∶乃阳气盛不得入于阴，阴气虚，故目不得瞑。治法饮以半夏 汤，用流水千里外者八升...
14                                                  NaN
15    旧八，新二十一。 九窍出血∶方见主治下。 衄血不止∶叶氏用新汲水，随左右洗足即止，累用有效。...
Name: 附方, dtype: object

In [38]:
import re
import jieba

jieba.set_dictionary('./jieba_data/dict.txt.big')
jieba.load_userdict('./jieba_data/userdict.txt')

Building prefix dict from D:\TMU_BigData\0.專題\程式\project(medicine)\medicine\medicine\test\jieba_data\dict.txt.big ...
Dumping model to file cache C:\Users\leoch\AppData\Local\Temp\jieba.u74bff60d95b469fcc55ef85938850064.cache
Loading model cost 1.483 seconds.
Prefix dict has been built succesfully.


In [105]:
signs = ''
with open('./jieba_data/punctuation.txt', encoding='utf-8') as f:
    signs = f.readlines()
    signs = [sign.strip() for sign in signs] 

signs_str = '|'.join(signs)
signs_str = re.sub('(\[|\])', r'\\\1', signs_str)
signs_str

'\\[|.|!|/|-|>|<|$|%|^|*|"|?|\\]|||\\[|—|！|，|。|？|、|（|）|：|」|「|)|(|【|】|╱|；|\\]|=|「|」|‧|（|）|.|~|《|》|∶'

In [106]:
sentences = []

for compound in compounds:
    if pd.isnull(compound):
        continue
    lines = re.split(',|，|。|;|【|】|\n|\u3000', compound)
    lines = [re.sub(' ', '', line) for line in lines]
    lines = [re.sub('['+signs_str+']', ' ', line) for line in lines]
    lines = [line.strip() for line in lines if line.strip() != '']
    # print(compound)
    # print(lines)
    sentences.append(lines)
sentences[0:10]

[['新一', '寒热疟疾 秋后霜一钱半', '热酒服之', '集玄方'],
 ['新一', '灭瘢痕 以冻凌频熨之', '良', '千金方'],
 ['旧一',
  '新一',
  '辟禳时疫 半天河水',
  '饮之',
  '医林集要  身体白驳 取树木孔中水洗之',
  '捣桂末唾和敷之',
  '日再上',
  '张文仲 备急方'],
 ['新三',
  '目不得瞑 乃阳气盛不得入于阴',
  '阴气虚',
  '故目不得瞑',
  '治法饮以半夏汤',
  '用流水千里外者八升',
  '扬之万遍',
  '取其清五升煮之',
  '炊苇薪火',
  '置秫米一升',
  '半夏五合',
  '徐炊令竭为一升半',
  '去滓饮汁一小杯',
  '日三饮',
  '以知为度',
  '详半夏下',
  '灵枢经  汗后奔豚 茯苓桂枝甘草大枣汤',
  '治发汗后脐下悸',
  '欲作奔豚者',
  '茯苓一两',
  '炙甘草二钱半',
  '桂枝三钱',
  '大枣二枚',
  '以甘澜水二升',
  '先煮茯苓',
  '纳诸药煮服之',
  '日三',
  '张仲景 金匮要略  服药过剂烦闷 东流水饮一 二升',
  '肘后方'],
 ['旧八',
  '新二十一',
  '九窍出血 方见主治下',
  '衄血不止 叶氏用新汲水',
  '随左右洗足即止',
  '累用有效',
  '一方 用冷水面',
  '一方 冷水浸纸贴囟上',
  '以熨斗熨之',
  '立止',
  '一方 用冷水一瓶',
  '淋射顶上及哑门上',
  '或以湿纸贴之',
  '金疮血出不止 冷水浸之即止',
  '延寿方  犬咬血出 以水洗',
  '至血止',
  '绵裹之',
  '千金方  蝎虿螫伤 以水浸故布拓之',
  '暖即易',
  '千金方  马汗入疮或马毛入疮',
  '肿入腹',
  '杀人',
  '以冷水浸之',
  '频易水',
  '仍饮好酒',
  '立瘥',
  '千金方  鱼骨哽咽 取水一杯',
  '合口向水',
  '张口取水气',
  '哽当自下',
  '肘后方  中砒石毒 多饮新汲井水',
  '得吐利佳',
  '集简方  中乌喙毒 方同上',
  '中蒙汗毒 饮冷水即安',
  '济急方  中煤炭毒 一时晕倒',
  '

In [103]:
terms = set()

for sentence in sentences:
    for line in sentence:
        segResult = jieba.cut(line, cut_all=False)
        print(list(segResult))
        terms = terms.union(set(segResult))

# stopwords = pd.read_csv('./data/stopword_chinese.txt', header=None)
# stopwords = stopwords[0].values.tolist()

# terms = terms.difference(set(stopwords))
# list(terms)[0:5]

['新一']
['寒热', '疟疾', ' ', '秋后', '霜', '一钱', '半']
['热酒服', '之']
['集玄方']
['新一']
['灭', '瘢痕', '∶', '以冻', '凌频', '熨', '之']
['良']
['千金方']
['旧', '一']
['新一']
['辟', '禳', '时疫', '∶', '半天', '河水']
['饮', '之']
['医林', '集要', ' ', ' ', '身体', '白驳', '∶', '取', '树木', '孔中', '水洗', '之']
['捣桂末', '唾', '和', '敷', '之']
['日', '再', '上']
['张文仲', ' ', '备急方']
['新', '三']
['目', '不得', '瞑', '∶', '乃', '阳气', '盛', '不得', '入于', '阴']
['阴气', '虚']
['故目', '不得', '瞑']
['治法', '饮以', '半夏', '汤']
['用', '流水', '千里', '外者', '八升']
['扬之万遍']
['取其', '清五升', '煮', '之']
['炊苇', '薪火']
['置', '秫米', '一升']
['半夏', '五合']
['徐炊', '令竭', '为', '一升半']
['去', '滓', '饮汁', '一', '小杯']
['日三饮']
['以知', '为度']
['详', '半夏', '下']
['灵枢经', ' ', ' ', '汗后', '奔豚', '∶', '茯苓', '桂枝', '甘草', '大枣', '汤']
['治', '发汗', '后', '脐', '下悸']
['欲作', '奔豚', '者']
['茯苓', '一两']
['炙', '甘草', '二钱', '半']
['桂枝', '三钱']
['大枣', '二枚']
['以甘澜', '水二升']
['先煮', '茯苓']
['纳', '诸药', '煮服', '之']
['日', '三']
['张仲景', ' ', '金匮要略', ' ', ' ', '服药', '过剂', '烦闷', '∶', '东', '流水', '饮一', ' ', '二升']
['肘后方']
['旧八']
['新', '二十一']
['九窍', '出血', '∶

['积德', '堂方', ' ', ' ', '消渴', '引饮', '∶', '瓦窑', '突上', '黑', '煤']
['干似', '铁', '屎', '者']
['半斤']
['为', '末']
['入', '生姜', '四两']
['同', '捣']
['绢袋', '盛']
['水五升', '浸汁']
['每', '饮五合']
['圣济录', ' ', ' ', '胞衣', '不下', '∶', '灶', '突后', '黑土', '三指', '撮']
['五更', '酒下']
['陈', '藏器']
['旧十']
['新六']
['吐血', '不止', '∶', '金墨', '磨汁']
['同', '莱菔', '汁', '饮']
['或', '生地黄', '汁', '亦可']
['集简方', ' ', ' ', '衄', '血', '不止']
['眩', '冒', '欲', '死', '∶', '浓', '墨汁', '滴入', '鼻中']
['梅师方', ' ', ' ', '热病', '衄', '血出', '数升者', '∶', '取好', '墨为', '末']
['鸡子', '白丸', '梧子', '大']
['用', '生地黄', '汁', '下', '一', ' ', '二十', '丸']
['少顷', '再服']
['仍以', '葱', '汁', '磨墨']
['滴入', '鼻内']
['即止']
['外台秘要', ' ', ' ', '大小便', '血', '∶', '好', '墨细末', '二钱']
['阿胶', '化汤', '调服']
['热', '多者', '尤', '相宜']
['寇氏', ' ', '本草', '衍义', ' ', ' ', '猝淋', '不通', '∶', '好墨', ' ', '烧', ' ', '一两']
['为', '末']
['每服', '一字']
['温水', '服之']
['普济方', ' ', ' ', '赤白', '下痢', '∶', '姜墨丸', '∶', '用', '干姜', ' ', '好墨', '各', '五两']
['为', '末']
['醋浆', '和', '丸', '梧子', '大']
['每服', '三', ' ', '四十', '丸']
['米', '饮下']
['日夜', '六',

['旧二']
['新八']
['服', '石英', '法', '∶', '白石英', '一斤']
['打成', '豆大']
['于', '砂', '盆中', '和', '粗砂']
['着', '水', '二三千', '下']
['洗净', '又']
['仍安柳', '箕', '中']
['入蒿', '叶', '少许']
['同水熟', '至', '光净']
['即以', '绵袋', '盛']
['悬', '门上']
['每日', '未', '梳前']
['以水', '或', '酒', '吞', '七粒']
['用饭', '二匙', '压下', '小腹']
['一切', '秽恶', ' ', '白酒', ' ', '牛肉']
['石家', '所忌者']
['皆', '不', '忌']
['久', '则', '新石', '推出', '陈石']
['石常', '在', '小腹', '内', '温暖']
['则', '气息', '调和']
['经脉', '通达']
['腰', '肾', '坚强']
['百病', '自除']
['石若', '得力']
['一斤', '即止', ' ', '若', '不', '得力']
['十斤', '亦须', '服']
['此物', '光滑']
['既', '无', '浮碎', '着', '人', '肠胃', '作疮']
['又', '无', '石气', '发作', '诸病', '也']
['又', '法', '∶', '泽州', '白石英']
['光净', '无点', '翳', '者']
['打', '小豆', '大']
['去', '细者']
['水淘净']
['袋盛']
['悬铛', '内']
['清水', '五大', '升']
['煮汁', '一升']
['澄清']
['平早', '服']
['以汁', '煮粥', '更佳']
['服后', '饮酒', '三', ' ', '二杯']
['可行', '百步']
['一袋', '可', '煮', '二十度']
['如', '无力']
['以布', '裹', '埋南墙', '下', '三尺', '土内']
['百日', '又', '堪用', '也']
['石煮', '猪肉', '法', '∶', '白石英', '一两']
['袋盛']
['水', '三斗']
['煮', '四升', ' '

['绵裹', '箸', '头', '填末入', '疮口']
['数次', '愈']
['简便', '方', ' ', ' ', '天泡', '湿疮', '∶', '无名异', '末']
['井华水', '调服', '之']
['普济方', ' ', ' ', '疮', '溃烂', '∶', '无名异', ' ', '虢丹', '细研']
['清油', '调', '搽']
['湿则', '干', '搽', '之']
['济急', '方', ' ', ' ', '股阴', '∶', '无名异', '二钱']
['麝香', '一字']
['研']
['酒', '半碗']
['午后', '空腹服']
['立效']
['多能', '鄙事', ' ', ' ', '拳毛', '倒睫', '∶', '无名异', '末']
['纸卷', '作', '捻']
['点灯', '吹', '杀', '熏', '之']
['睫自起']
['保命', '集', ' ', ' ', '消渴', '引饮', '∶', '无名异', '一两']
['黄连', '二两']
['为', '末']
['蒸饼', '丸', '绿豆', '大']
['每服', '百丸']
['以茄根', ' ', '蚕茧', '煎', '汤', '送', '下']
['圣济录', ' ', ' ', '香港脚', '痛楚', '∶', '无名异', '末']
['化', '牛皮胶', '调涂', '之']
['频换']
['卫生', '易', '简方']
['新', '十一']
['李', '补阙', '服乳法', '∶', '主', '五劳七伤']
['咳逆上', '气']
['治寒嗽']
['通音声']
['明目', '益精']
['安', '五脏']
['通', '百节']
['利', '九窍']
['下', '乳汁']
['益气', '补', '虚损']
['疗脚弱', '疼', '冷']
['下焦', '伤竭']
['强阴']
['久服', '延年益寿', '不老']
['令人', '有子']
['取', '韶州', '钟乳']
['无问', '浓薄']
['但', '颜色', '明净', '光泽', '者']
['即堪入', '炼', ' ', '惟黄', ' ', '赤二色']
['不', '任用']
['置于

['服', '石发', '疮', '疼', '不可', '忍', '∶', '用纸', '圈围', '之']
['中心', '填', '硝石', '令满']
['以匙', '抄', '水淋', '之']
['觉不热痛']
['即止']
['兵部', '手集', ' ', ' ', '发背', '初起', '∶', '恶寒', '啬', '啬']
['或', '已', '生疮', '肿瘾疹']
['硝石', '三两']
['暖水', '一升']
['泡化']
['待', '冷']
['青布', '折', '三重']
['湿拓', '赤处']
['热即', '换']
['频易', '取', '瘥']
['外台秘要', ' ', ' ', '女劳', '黑疸', '∶', '仲景', '曰', '∶', '黄家', '日', '晡', '发热']
['反', '恶寒']
['此为', '女', '劳得', '之']
['膀胱', '急']
['小腹', '满']
['身尽', '黄']
['额上', '黑']
['足下', '热']
['因作', '黑疸']
['腹胀', '如水']
['大便', '黑']
['时', '溏']
['非水', '也']
['腹满者', '难治']
['硝石', ' ', '矾石', ' ', '烧', ' ', '等', '分']
['为', '末']
['以大', '麦粥', '汁', '和服', '方寸', '匕']
['日', '三']
['病随', '大小便', '去']
['小便', '黄']
['大便', '黑']
['是', '其候', '也']
['金匮', ' ', ' ', '手足', '不遂', '大风']
['及丹石', '热风', '不遂']
['用', '硝石', '一两']
['生乌', '麻油', '二斤']
['置铛', '中']
['以土堑', '盖口']
['纸泥', '固济']
['火', '煎']
['初时', '气腥']
['熟则', '气香']
['更', '以生', '麻油', '二升']
['合', '煎', '得', '所']
['收不津器', '中']
['服时', '坐室', '中']
['重作', '小纸', '屋']
['燃火于', '内']
['服', '一大', '合']
[

...

['研匀']
['温酒', '调服']
['须臾', '红活']
['神效']
['无干血']
['用生', '血']
['沈存', '中方', ' ', '妇人', '催生', '∶', '开骨膏', '∶', '用', '猪', '心血', '和', '乳香', '末']
['丸', '梧桐子', '大']
['朱砂', '为衣']
['面东', '酒', '吞', '一丸']
['未下', '再服']
['妇人', '良方']
['旧', '一']
['新一']
['猝中恶', '死', '∶', '断', '猪尾', '取', '血饮']
['并缚豚', '枕', '之']
['即活']
['此乃长', '桑君授', '扁鹊', '法', '也']
['出', ' ', '魏', '夫人', '传']
['肘后方', ' ', ' ', '蛇入', '七孔', '∶', '割', '母猪', '尾血']
['滴入', '即出', '也']
['千金方']
['旧', '一']
['新', '三']
['个']
['带血', '破开']
['入', '人参', ' ', '当归', '各', '二两']
['煮熟', '去', '药食', '之']
['不过', '数服']
['即', '愈']
['证治', '要诀', ' ', ' ', '心虚', '嗽血', '∶', '沉香', '末', '一钱']
['半夏', '七枚']
['入猪', '心中']
['以', '小便', '湿', '纸包', '煨', '熟']
['去', '半夏', '食', '之']
['证治', '要诀', ' ', ' ', '产后', '风邪']
['心虚', '惊悸', '∶', '用', '猪心', '一枚']
['五味']
['豉汁', '煮食', '之']
['心镜', ' ', ' ', '急心', '疼痛', '∶', '猪心', '一枚']
['每', '岁入', '胡椒', '一粒']
['同盐', ' ', '酒', '煮食']
['旧七']
['新七']
['休息', '痢疾', '∶', '猪肝', '一具', ' ', '切片']
['杏仁', ' ', '炒', ' ', '一两']
['于净', '锅', '内']
['一重', '肝']
['

['五劳七伤']
['用', '羊', '肾', '三', '对']
['羊肉', '半斤']
['葱白', '一茎']
['枸杞', '叶', '一斤']
['同', '五味', '煮成', '汁']
['下米作', '粥', '食', '之']
['虚损', '劳伤', '∶', '羊', '肾', '一枚']
['术', '一升']
['水', '一斗']
['煮', '九升']
['服一升']
['日', '三']
['肘后方', '肾虚', '腰痛', '∶', ' ', '千金', ' ', '∶', '用', '羊', '肾', '去', '膜']
['阴干', '为', '末']
['酒服', '二', '方寸', '匕']
['日', '三']
['正要', ' ', '∶', '治猝', '腰痛']
['羊', '肾', '一对']
['咱夫兰', '一钱']
['玫瑰', '水', '一盏', '浸汁']
['入盐', '少许']
['涂抹', '肾上']
['徐徐', '炙熟']
['空腹', '食', '之']
['老人', '肾硬', '∶', '治', '老人', '肾脏', '虚寒']
['内', '肾', '结硬']
['虽服', '补药', '不', '入']
['用羊', '肾子', '一对']
['杜仲', ' ', '长', '二寸']
['阔', '一寸', ' ', '一片']
['同', '煮熟']
['空心', '食', '之']
['令人', '内', '肾', '柔软']
['然后', '服', '补药']
['鸡峰备', '急方', ' ', ' ', '胁', '破肠出', '∶', '以', '香油', '抹手', '送入']
['煎', '人参', ' ', '枸杞子']
['旧四']
['新', '十一']
['目热', '赤痛']
['看物', '如', '隔纱', '∶', '宜补', '肝益睛']
['用青', '羊肝', '一具', '切洗']
['和', '五味', '食', '之']
['心镜', ' ', ' ', '肝虚目', '赤', '∶', '青', '羊肝']
['薄切', '水浸']
['吞之极效']
['龙木论', ' ', ' ', '病后', '失明', '∶', '方同

['以水研']
['绞汁', '一升服']
['梅师方', ' ', ' ', '衄', '血', '不止', '∶', ' ', '录验', ' ', '∶', '用绵裹', '白马', '屎', '塞', '之']
['千金', ' ', '∶', '用', '赤', '马粪', '绞汁']
['干者', '浸水', '亦可']
['口', '鼻出血', '∶', '用', '赤', '马粪', '烧灰']
['温酒', '服一钱']
['钤方', ' ', ' ', '久', '痢', '赤白', '∶', '马粪', '一丸', '烧灰']
['水服']
['肘后方', ' ', ' ', '猝中恶', '死吐利', '不止']
['不知', '是', '何病']
['不拘', '大人', '小儿', '∶', '马粪', '一丸']
['绞汁', '灌', '之']
['干者', '水', '煮汁', '亦可']
['此', '扁鹊', '法', '也']
['肘后', ' ', ' ', '搅肠', '沙痛', '欲', '死者', '∶', '用', '马粪', '研汁', '饮', '之']
['立愈']
['经验', '方', ' ', ' ', '小儿', '猝', '忤', '∶', '马屎', '三升', '烧末']
['以酒', '三斗']
['煮', '三沸']
['取汁', '浴儿']
['避风']
['千金', ' ', ' ', '小儿', '啼', '∶', '面青', '腹强']
['是', '忤', '客气']
['新', '马粪', '一团']
['绞汁', '灌', '之']
['伤寒', '劳复', '∶', '马屎', '烧末']
['冷酒', '服', '方寸', '匕']
['便验']
['外台', ' ', ' ', '热毒', '攻肢']
['手足', '肿痛', '欲', '脱', '∶', '以水', '煮', '马', '屎', '汁', '渍', '之']
['外台', ' ', ' ', '风虫', '牙痛', '∶', '白马', '屎', '汁']
['随', '左右', '含', '之']
['不过', '三口', '愈']
['圣惠', ' ', ' ', '鼻不闻', '∶', '新马', 

['理腰', '脚']
['能久立']
['聪耳', '明目']
['发白', '更', '黑']
['貌', '老', '还', '少']
['凡', '麋角']
['取', '当年', '新角', '连脑', '顶者', '为', '上']
['看角', '根有', '斫', '痕处']
['亦', '堪用']
['蜕角', '根下', '平者']
['不堪']
['取角', '五具']
['或', '四具', ' ', '三具', ' ', '二具', ' ', '一具', '为', '一剂']
['去', '尖', '一大', '寸']
['即角长', '七八寸']
['取势', '截断']
['量', '把', '镑', '得']
['即于', '长流水', '软', '即将', '出']
['削去', '皱皮']
['以利', '镑', '镑', '取白处']
['至心', '即止']
['以清', '粟米', '泔浸', '两宿']
['初经', '一宿', '即', '干']
['握沥去', '旧水']
['置', '新', '绢', '上', '曝干']
['择去', '恶物', '粗骨皮', '及', '镑', '不', '匀者']
['以无灰', '美酒', '于', '大', '瓷器', '中浸']
['经两宿']
['其药', '及', '酒俱入', '净', '釜', '中']
['初用', '武火', '煮', '一食', '久']
['后以', '文火', '微', '煎']
['如蟹', '目沸']
['以', '柳木', '篦', '徐徐', '搅']
['不得', '住手']
['时时', '添酒']
['以成', '煎', '为度']
['煎', '时皆须', '平旦', '下手']
['不得', '经宿']
['仍', '看屑', '消如', '稀胶']
['即以', '牛乳', '五升']
['酥', '一斤']
['以次', '渐下', '后', '项药']
['仍以', '麋角', '一条']
['炙令', '黄为', '末']
['与', '诸药', '同制', '之']
['槟榔', ' ', '通草', ' ', '秦艽', ' ', '肉苁蓉', ' ', '人参', ' ', '菟丝子', ' ', '酒浸'

['未汗', '再服']
['南阳', '活人', '方', ' ', ' ', '大小', '便秘', '∶', '雄鼠', '屎', '末']
['敷脐中']
['立效']
['普济', ' ', ' ', '室女', '经闭', '∶', '牡鼠', '屎', '一两', '炒研']
['空心', '温酒', '服半钱']
['千金方', ' ', ' ', '子死', '腹中', '∶', '雄鼠', '屎', '二七', '枚']
['水三升']
['煮', '一升']
['取汁作', '粥', '食']
['胎即', '下']
['子母', '秘录', ' ', ' ', '产后', '阴脱', '∶', '以', '温水', '洗软']
['用雄鼠', '屎', '烧', '烟熏', '之', '即入']
['熊氏', ' ', '妇人', '吹奶', '∶', '鼠', '屎', '七粒']
['红枣', '七枚', '去核', '包屎']
['烧存性']
['入', '麝香', '少许']
['温酒', '调服']
['集要', ' ', '方', ' ', '乳痈初', '起', '∶', '雄鼠', '屎', '七枚', '研末']
['温酒', '服']
['取汗', '即', '散']
['寿域', '方', ' ', ' ', '乳', '痈', '已成', '∶', '用', '新', '湿鼠', '屎', ' ', '黄连', ' ', '大黄', '各', '等', '分为', '末']
['以', '黍', '米粥', '清', '和']
['涂', '四边']
['即散']
['姚僧', '坦方', ' ', '鼠溃', '坏', '∶', '新鼠', '屎', '一百', '粒']
['收密器', '中', '五六十', '日']
['杵', '碎']
['即', '敷', '之']
['效']
['千金方', ' ', ' ', '疔', '疮', '恶肿', '∶', '鼠', '屎', ' ', '乱发', '等', '分', '烧灰']
['针疮头', '纳入']
['大良']
['普济方', ' ', ' ', '鬼击', '吐血']
['胸腹', '刺痛', '∶', '鼠', '屎', '烧末']
['水服', 

In [90]:
len(terms)

26651

In [92]:
# terms

In [111]:
compounds[6]

'新一。寒热疟疾（秋后霜一钱半，热酒服之。《集玄方》） '

In [163]:
num_str = r'[一二三四五六七八九十]'
# pattern = r'(旧' + num_str + r'{1,3})|(新' + num_str + '{1,3})'
pattern = r'^(旧' + num_str + r'{1,3}|新' + num_str + r'{1,3}.*?。)'
re.search(pattern, compounds[6]).group()

'新一。'

In [171]:
num_str = r'[一二三四五六七八九十]'
pattern = r'((旧' + num_str + r'{1,3})?(新' + num_str + r'{1,3})?.*?。)'
for c in compounds:
    if pd.isnull(c):
        continue
    match = re.search(pattern, c)
    if match:
        pass
        # print(match.group())
    else:
        print(c)


（原缺） 
（原缺） 
